advanced dynamic validation test - make parameter validation depend on the value of other parameters

In [1]:
import jsonschema

In [2]:
schema = {
    "type": "object",
    "properties": {
        "payment method": {
            "type": "string",
            "enum": ["transfer", "PyPal", "credit card"]
        },
        "card number": {"type": "string"},
        "cardholder name": {"type": "string"}
    },
    "required": ["payment method"],
    "anyOf": [
        {
            "properties": {
                "payment method": {"const": "credit card"}
            },
            "required": ["card number", "cardholder name"]
        },
        {
            "not": {
                "anyOf": [
                    {"properties": {
                        "payment method": {"const": "credit card"}
                    }},
                    {"required": ["card number"]},
                    {"required": ["cardholder name"]}
                ]
            }
        }
    ],
    "additionalProperties": False
}
validator = jsonschema.Draft7Validator(schema)

In [3]:
# since draft7 there is an "if" statement
schema = {
    "type": "object",
    "properties": {
        "payment method": {
            "type": "string",
            "enum": ["transfer", "PyPal", "credit card"]
        },
        "card number": {"type": "string"},
        "cardholder name": {"type": "string"}
    },
    "required": ["payment method"],
    "if": {
        "properties": {
            "payment method": {"const": "credit card"}
        }
    },
    "then": {
        "required": ["card number", "cardholder name"]
    },
    "else": {
        "allOf": [
            {"not": {"required": ["card number"]}},
            {"not": {"required": ["cardholder name"]}}
        ]
    },
    "additionalProperties": False
}
validator_alt = jsonschema.Draft7Validator(schema)

In [4]:
instance = {
    "payment method": "PyPal"
}
validator.validate(instance)
validator_alt.validate(instance)

In [5]:
instance = {
    "payment method": "credit card",
    "card number": "1234-5678-8765-4321",
    "cardholder name": "Steve McSmith"
}
validator.validate(instance)
validator_alt.validate(instance)

In [6]:
instance = {
    "payment method": "transfer",
    "card number": "1234-5678-8765-4321"
}
print(list(validator.iter_errors(instance)))
print(list(validator_alt.iter_errors(instance)))

[<ValidationError: "{'payment method': 'transfer', 'card number': '1234-5678-8765-4321'} is not valid under any of the given schemas">]
[<ValidationError: "{'payment method': 'transfer', 'card number': '1234-5678-8765-4321'} should not be valid under {'required': ['card number']}">]


In [7]:
instance = {
    "payment method": "transfer",
    "card number": "1234-5678-8765-4321",
    "cardholder name": "Steve McSmith"
}
print(list(validator.iter_errors(instance)))
print(list(validator_alt.iter_errors(instance)))

[<ValidationError: "{'payment method': 'transfer', 'card number': '1234-5678-8765-4321', 'cardholder name': 'Steve McSmith'} is not valid under any of the given schemas">]
[<ValidationError: "{'payment method': 'transfer', 'card number': '1234-5678-8765-4321', 'cardholder name': 'Steve McSmith'} should not be valid under {'required': ['card number']}">, <ValidationError: "{'payment method': 'transfer', 'card number': '1234-5678-8765-4321', 'cardholder name': 'Steve McSmith'} should not be valid under {'required': ['cardholder name']}">]


In [8]:
instance = {
   "payment method": "credit card" 
}
print(list(validator.iter_errors(instance)))
print(list(validator_alt.iter_errors(instance)))

[<ValidationError: "{'payment method': 'credit card'} is not valid under any of the given schemas">]
[<ValidationError: "'card number' is a required property">, <ValidationError: "'cardholder name' is a required property">]
